<a href="https://colab.research.google.com/github/oneHAMazing/-HAM-PSMDSRC103/blob/main/HAM_Seatwork_8_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQLite3 in Google Colab

SQLite is a lightweight, file-based SQL database engine that requires no separate server process. It is self-contained and zero-configuration, meaning you can use it out of the box without installing or configuring a database server. SQLite databases are stored in a single file on disk (often with a .db extension), making them easy to move or version-control.

Despite its simplicity, SQLite is ACID-compliant and supports most SQL features (tables, indexes, transactions, triggers, etc.), making it ideal for prototyping, embedded applications, and local data analysis.

## Setting up SQLite3

Google Colab runs Python in a cloud VM, and Python’s built-in `sqlite3` module is available by default. You can start using SQLite without installing anything extra. Just import the library and create (or connect to) a database file. For example:

In [1]:
import sqlite3
# Connect to a (new) SQLite database file; this creates 'data.db' in the working directory.
conn = sqlite3.connect('data.db')
cur = conn.cursor()

* This `connect() call either opens an existing file (data.db) or creates it if it doesn’t exist.

* The returned conn object represents the database connection (backed by the file), and `conn.cursor()` creates a cursor for executing SQL commands.

* By default SQLite is file-based, so you will see data.db in the Colab file browser after creation. (If you need persistent storage across sessions, you can mount Google Drive with from google.colab import drive and save the file there.)

Tip: As an alternative, Colab supports the %%sql cell magic (from ipython-sql) to run SQL commands in cells. For example %load_ext sql then %%sql sqlite:///data.db. However, this tutorial focuses on using the sqlite3 Python API.

* After creating a connection, you can execute SQL commands via cur.execute(...) and fetch results.

An example on how to create tables and run queries:
`cur.execute("CREATE TABLE movie(title, year, score)")`
This creates a new table. In SQLite, column types are flexible, so you can list column names without strict types if desired.

## Basic CRUD Operations (Creating, Reading, Updating, Deleting)

### Creating Tables

To create a new table in the database, use the SQL CREATE TABLE statement.

In [2]:
# Create a new table named 'students' with 3 columns.
cur.execute("""
CREATE TABLE IF NOT EXISTS students (
    id INTEGER PRIMARY KEY,
    name TEXT,
    grade REAL
)
""")

conn.commit()  # Save (commit) the transaction

This code creates a table students(id, name, grade) if it doesn’t already exist. Notice the call to `conn.commit()` in SQLite, DDL and DML statements (like CREATE, INSERT, UPDATE, DELETE) are executed within a transaction.

The changes are not saved to the database file until you commit. The official docs state that INSERT (or other write operations) "implicitly opens a transaction, which needs to be committed before changes are saved in the database".

Calling conn.commit() flushes the transaction. (If you do not call commit(), the changes will be lost when the connection closes.)

### Inserting Data

Once you have a table, insert data with `INSERT INTO`. For example:

In [3]:
# Insert a single row (note use of placeholders ?)
cur.execute("INSERT INTO students (name, grade) VALUES (?, ?)",
            ("Alice", 91.5))

conn.commit()

Using `?` placeholders is strongly recommended (instead of Python string formatting) for safety and correctness. Placeholders prevent SQL injection and ensure values are quoted properly. You pass a tuple of values as the second argument to execute.

The Python docs caution: “Always use placeholders instead of string formatting to bind Python values to SQL statements, to avoid SQL injection attacks”.

For bulk inserts, use `executemany()`. For example:

In [4]:
data = [
    ("Bob",   85.0),
    ("Carol", 92.0),
    ("Dave",  88.5),
]

cur.executemany("INSERT INTO students (name, grade) VALUES (?, ?)", data)
conn.commit()

Here `executemany()` inserts multiple rows in one call. Doing many inserts inside a single transaction (as shown) is much faster than committing each row separately. The Python tutorial example does exactly this with two lists of movie data and advises committing once after `executemany()`.

### Querying and Fetching Data

To read data, use `SELECT`. For example:

In [5]:
# Query all students
cur.execute("SELECT * FROM students")
rows = cur.fetchall()
for row in rows:
    print(row)

(1, 'Alice', 91.5)
(2, 'Bob', 85.0)
(3, 'Carol', 92.0)
(4, 'Dave', 88.5)


`cur.fetchall()` returns a list of tuples, one tuple per row. Each tuple contains the column values for that row. For example, after inserting three students above, `fetchall()` might return:

```
[(1, 'Alice', 91.5), (2, 'Bob', 85.0), (3, 'Carol', 92.0), (4, 'Dave', 88.5)]
```

where the first element is the `id` (auto-incremented), and the others are name and grade. You can also fetch rows one by one with `fetchone()` or iterate directly: `for row in cur.execute("SELECT name FROM students"):`.

You can use SQL clauses like `WHERE, ORDER BY, GROUP BY` as usual. For example:

In [6]:
# Get the average grade (per student or overall):
cur.execute("SELECT AVG(grade) FROM students")
avg = cur.fetchone()[0]
print("Average grade:", avg)

Average grade: 89.25


### Updating and Deleting Data

Use `UPDATE` and `DELETE` for modifying data. For example:

In [7]:
# Give Bob a new grade
cur.execute("UPDATE students SET grade = ? WHERE name = ?", (89.0, "Bob"))
conn.commit()

# Delete a row
cur.execute("DELETE FROM students WHERE name = ?", ("Dave",))
conn.commit()

Always use `?` placeholders for user-supplied values. After each `UPDATE` or `DELETE`, call `conn.commit()` to save changes. **(If an error occurs, you can call conn.rollback() to undo the current transaction before committing.)**

In [8]:
cur.execute("Select * from students")
rows2 = cur.fetchall()
for row in rows2:
  print(row)

(1, 'Alice', 91.5)
(2, 'Bob', 89.0)
(3, 'Carol', 92.0)


## Importing and Exporting Data

### Loading CSV Data into SQLite

Often you’ll want to import external data (e.g. from CSV) into SQLite. In Colab, you can use shell commands or Python to download datasets. For example, to download a CSV:

In [9]:
!wget -O iris.csv https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv

--2025-05-04 04:04:10--  https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv
Resolving gist.githubusercontent.com (gist.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to gist.githubusercontent.com (gist.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3975 (3.9K) [text/plain]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   3.88K  --.-KB/s    in 0s      

2025-05-04 04:04:10 (44.1 MB/s) - ‘iris.csv’ saved [3975/3975]



Then load it with Python. A straightforward way is to use Python’s csv module:

In [10]:
import csv

In [11]:
cur.execute("CREATE TABLE IF NOT EXISTS people (sepal_length FLOAT, sepal_width FLOAT, petal_length FLOAT, petal_width FLOAT, variety TEXT)")

In [12]:
with open('iris.csv', 'r') as f:
    reader = csv.reader(f)
    header = next(reader)  # skip header row if present
    rows = [tuple(row) for row in reader]

In [13]:
rows

[('5.1', '3.5', '1.4', '.2', 'Setosa'),
 ('4.9', '3', '1.4', '.2', 'Setosa'),
 ('4.7', '3.2', '1.3', '.2', 'Setosa'),
 ('4.6', '3.1', '1.5', '.2', 'Setosa'),
 ('5', '3.6', '1.4', '.2', 'Setosa'),
 ('5.4', '3.9', '1.7', '.4', 'Setosa'),
 ('4.6', '3.4', '1.4', '.3', 'Setosa'),
 ('5', '3.4', '1.5', '.2', 'Setosa'),
 ('4.4', '2.9', '1.4', '.2', 'Setosa'),
 ('4.9', '3.1', '1.5', '.1', 'Setosa'),
 ('5.4', '3.7', '1.5', '.2', 'Setosa'),
 ('4.8', '3.4', '1.6', '.2', 'Setosa'),
 ('4.8', '3', '1.4', '.1', 'Setosa'),
 ('4.3', '3', '1.1', '.1', 'Setosa'),
 ('5.8', '4', '1.2', '.2', 'Setosa'),
 ('5.7', '4.4', '1.5', '.4', 'Setosa'),
 ('5.4', '3.9', '1.3', '.4', 'Setosa'),
 ('5.1', '3.5', '1.4', '.3', 'Setosa'),
 ('5.7', '3.8', '1.7', '.3', 'Setosa'),
 ('5.1', '3.8', '1.5', '.3', 'Setosa'),
 ('5.4', '3.4', '1.7', '.2', 'Setosa'),
 ('5.1', '3.7', '1.5', '.4', 'Setosa'),
 ('4.6', '3.6', '1', '.2', 'Setosa'),
 ('5.1', '3.3', '1.7', '.5', 'Setosa'),
 ('4.8', '3.4', '1.9', '.2', 'Setosa'),
 ('5', '3', '1

In [14]:
cur.executemany("INSERT INTO people VALUES (?, ?, ?, ?, ?)", rows)
conn.commit()

This reads rows from `iris.csv` into a list of tuples, then inserts them. (Adjust the CREATE TABLE columns to match your CSV structure.)

Alternatively, pandas makes this easier. You can simply `pd.read_csv()` and use `to_sql`.

### Exporting SQLite Data

You can also export query results to CSV or Pandas. For example, to write a query to CSV:

In [15]:
import csv
cur.execute("SELECT name, grade FROM students")
rows = cur.fetchall()

with open('grades.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerow([d[0] for d in cur.description])  # header
    writer.writerows(rows)

This writes query results to grades.csv. In Colab you can then download it or save it to Drive.

### SQLite and Pandas Integration

pandas provides convenient functions to move data between DataFrames and SQL.

* **DataFrame → SQL:** Use `DataFrame.to_sql()`. This writes the DataFrame to a database table. For example:

In [16]:
import pandas as pd

# Suppose df is a pandas DataFrame
df = pd.DataFrame({
    'id': [1,2,3],
    'name': ['Alice','Bob','Carol'],
    'score': [85, 92, 78]
})

df.to_sql('scores', conn, if_exists='replace', index=False)

3

* This creates (or replaces) the table scores with the DataFrame contents. As the pandas documentation states, “Write records stored in a DataFrame to a SQL database”.
* The `if_exists` parameter can be `'fail', 'replace', or 'append'` depending on whether you want to overwrite or add to an existing table.

* **SQL → DataFrame:** Use `pandas.read_sql()` (or `read_sql_query`). For example:

In [17]:
df_scores = pd.read_sql("SELECT name, score FROM scores WHERE score > 80", conn)

In [18]:
df_scores

,name,score
0,Alice,85
1,Bob,92


* This runs the SQL query and returns the results as a DataFrame `df_scores`. The pandas docs say “Read SQL query or database table into a DataFrame."
* This is very useful for analysis: you can use SQL to filter/aggregate data, then load into pandas for further processing.


Together, these features let you combine the efficiency of SQL for data loading/manipulation with pandas for analytics. For example, you might load a large CSV into an SQLite table, then use `pd.read_sql("SELECT ...", conn)` to retrieve just a subset or summary into a DataFrame.

## Joins and Relational Queries

One strength of SQL is joining multiple tables. In SQLite you can use standard `JOIN syntax`. For example, suppose we have two tables employees and departments:

In [20]:
cur.execute("CREATE TABLE dept(id INTEGER PRIMARY KEY, name TEXT)")
cur.execute("CREATE TABLE emp(id INTEGER PRIMARY KEY, name TEXT, deptid INTEGER)")

cur.executemany("INSERT INTO dept VALUES (?, ?)", [(1, "Sales"), (2, "IT")])
cur.executemany("INSERT INTO emp VALUES (?, ?, ?)",
                [(101, "Alice", 1), (102, "Bob", 2), (103, "Carol", 1)])
conn.commit()


To combine them, use an INNER JOIN:

In [21]:
query = """
SELECT e.name AS employee, d.name AS department
FROM emp AS e
INNER JOIN dept AS d ON e.deptid = d.id
"""

cur.execute(query)
for row in cur.fetchall():
    print(row)
# ('Alice', 'Sales'), ('Bob', 'IT'), ('Carol', 'Sales')

('Alice', 'Sales')
('Bob', 'IT')
('Carol', 'Sales')


This returns each employee with their department name. In general, SQLite supports **INNER JOIN, LEFT JOIN, CROSS JOIN,** etc., just like other SQL databases. You can use an INNER JOIN, LEFT JOIN, or CROSS JOIN clause to query data from two tables. (A LEFT JOIN would include departments even if no employee exists, and so on.)


Joins are powerful in many use cases; for example, you might have one table of experimental results and another of sample metadata, then join them by a key to analyze the combined data. Using `pd.read_sql` on join queries is also straightforward.

## Advanced SQLite3 Features

### Parameterized Queries (Placeholders)

We briefly touched on placeholders with `?`. Always use them for inserting or updating with external values. For example:

In [22]:
name = "Eve"
score = 77
cur.execute("INSERT INTO students (name, grade) VALUES (?, ?)", (name, score))

Do *not* do string formatting like `f"...VALUES ('{name}', {score})";` that can lead to SQL injection or quoting issues. The SQLite docs explicitly warn to use placeholders. Placeholders also handle proper escaping of quotes, types, etc.

### User-Defined Functions (UDFs)

SQLite allows you to define your own SQL functions in Python. This can be useful for custom computations. For example, you can register a Python function as an SQL function with `create_function`:

In [23]:
class MySum:
    def __init__(self):
        self.total = 0
    def step(self, value):
        self.total += value
    def finalize(self):
        return self.total

conn.create_aggregate("mysum", 1, MySum)
cur.execute("CREATE TABLE test(n)")
cur.executemany("INSERT INTO test(n) VALUES(?)", [(1,), (2,), (3,)])
cur.execute("SELECT mysum(n) FROM test")
print(cur.fetchone()[0])  # 6

6


This mirrors the Python docs’ aggregate example. User-defined aggregates let you implement custom reductions (beyond built-in SUM, AVG, etc.) within SQL queries.

In [24]:
conn.close()

Don't forget to close!

# Example: Full Iris Dataset Workflow

To illustrate a full workflow, let’s load a real dataset into SQLite and run some queries. We’ll use the classic Iris dataset from sklearn (150 flower measurements, 3 species). First, load it into a pandas DataFrame and write it to SQLite:

In [25]:
from sklearn.datasets import load_iris
import pandas as pd

iris = load_iris(as_frame=True)
df = iris.frame  # a DataFrame with sepal/petal lengths and target species (0,1,2)

conn = sqlite3.connect('iris.db')

df.to_sql('iris', conn, if_exists='replace', index=False)

150

Now the SQLite table iris has all 150 rows. We can query it directly or via pandas. For example, compute the average sepal length by species (target):

In [26]:
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [27]:
query = """
SELECT target AS species_id, AVG([sepal length (cm)]) AS avg_sepal
FROM iris
GROUP BY target
"""
result = pd.read_sql(query, conn)
print(result)

   species_id  avg_sepal
0           0      5.006
1           1      5.936
2           2      6.588


This returns a DataFrame like seen above.

Here `species_id 0,1,2` correspond to `Iris-setosa`, `Iris-versicolor`, and `Iris-virginica`. We see virginica has the longest sepals on average. Using `read_sql` demonstrates combining SQL aggregation with pandas.

We can also do a join. Suppose we have a lookup for species names:

In [28]:
species_df = pd.DataFrame({
    "target": [0, 1, 2],
    "species": ["setosa", "versicolor", "virginica"]
})

In [29]:
species_df.to_sql('species', conn, index=False)

query = """
SELECT s.species, AVG(i.[petal length (cm)]) AS avg_petal
FROM iris AS i
JOIN species AS s ON i.target = s.target
GROUP BY s.species
ORDER BY s.species
"""

df_join = pd.read_sql(query, conn)
print(df_join)


      species  avg_petal
0      setosa      1.462
1  versicolor      4.260
2   virginica      5.552


In [30]:
conn.close()

This join combines the `iris` table with our `species` lookup table. The result shows, for each species name, the average petal length. This exemplifies a small analytic project: you loaded a dataset into SQLite, ran SQL to summarize and join data, and pulled it into pandas.

Beyond Iris, you could scale up: for example, download a public dataset (e.g. COVID-19 stats from Our World in Data) directly into pandas or using `!wget`, and then `to_sql` it. Or load the MovieLens ratings dataset (millions of rows) into SQLite and perform SQL queries for analysis. The workflow is the same: **load data → run SQL → analyze**.

## Conclusion and Best Practices

In summary, using SQLite in Google Colab is straightforward because Python’s sqlite3 module is built in. Use it to create a local .db file, then run SQL commands with execute or via pandas. Remember to:

* **Commit transactions**: After any inserts/updates/deletes, call `conn.commit()` (or use a `with conn:` context).
* **Use placeholders**: Always use `?` parameters rather than formatting SQL strings.
* **Index wisely**: Add `CREATE INDEX` on columns you query frequently.
* **Leverage pandas**: Use `df.to_sql(...)` and `pd.read_sql(...)` to move data between pandas and SQLite.
* **Close connections**: When done, call `conn.close()` to free resources.

SQLite’s simplicity makes it an excellent tool for data science projects in Colab. You get the full power of SQL for filtering, joining, and aggregating data, combined with Python for processing and visualization. With these techniques, you can efficiently explore and analyze datasets entirely within a Colab notebook.

# References

* [About SQLite](https://www.sqlite.org/about.html)
* [sqlite3 — DB-API 2.0 interface for SQLite databases — Python 3.13.3 documentation](https://docs.python.org/3/library/sqlite3.html)
* [pandas.DataFrame.to_sql — pandas 2.2.3 documentation](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)
* [A Visual Explanation of SQLite Joins](https://www.sqlitetutorial.net/sqlite-join/)
* [Best practices for SQLite performance  |  App quality  |  Android Developers](https://developer.android.com/topic/performance/sqlite-performance-best-practices)

#Answers to Questions

1. Describe, in your own words, how explicit transaction control (BEGIN…COMMIT, rollback(), or the with conn: context manager) affected both reliability and performance in your notebook workflow. Why is thoughtful use of transactions especially important when importing large, real-world datasets?
> Large, real- world datasets are almost always never clean. When explicit transaction control is used, then any changes to clean, structure, and enrich our real-world data can be well-documented and repeatable. Explicit transaction control means that all data changes are treated as one operation. It makes it easier to undo changes (rollback) and makes committing changes easier by doing it in batches instead of per line of work. It also made reading the code easy since it was done in chunks instead of one big monster code that we had to put comments in between to ensure that we understood what was happening or what we think was happening.



2. Reflect on the difference between parameterized SQL statements (? placeholders) and string-formatted SQL. Beyond security, discuss how parameterization contributes to the reproducibility and long-term maintainability of data-science notebooks shared among collaborators.
>I honestly thought at first that parameter use in our codes has always been a comsci best practice. I have never thought too much about it. When there was a comment made on it during class, I was surprised that there was a deeper meaning behind it beyond coding quirks and ease of use. Now that I have thought on it a bit, I think parameterized SQL statements makes our lives easier. (See insert statement with five ? instead of typed out headers that could be exposed to typos or errors like forgetting to type one or more headers.) While string- formatted SQL can be done and might sometimes seem easier than parameterized SQL statements, the use of parameters makes the code clean, easier to read and debug, and enhances collaboration. When I said I initially thought it was a coding quirk, I thought that the use of parameters to make it easier to read and debug was just an ingrained habit to anyone who took comsci classes. Also, ensuring that someone else can easily understand and debug your code was something I thought was another habit picked up during the undergrad era to make sure that your groupmates or someone you pass on your codes to can easily understand and edit the code to make it look like their own.

3. Explain how moving data back and forth between pandas DataFrames and SQLite tables shaped your analytical workflow. What advantages did each tool provide, and how did their combination change the way you thought about data cleaning, aggregation, or visualization?
>While I cannot comment extensively on SQLite tables yet (I have only been introduced to it.), I can comment on the use of pandas DataFrames. In this notebook alone, the use of pandas makes it easier to understand since it is a language that we have used quite a bit. Also, if you put the data into a pandas Dataframe and work on it like that, we can use the full scope of features and convenience that pandas can provide. I think, if using one (pandas) makes things easier, using both (wisely) can make it much easier still since we can use combos that would make data handling great again.

4. Evaluate the practical benefits of registering Python functions (scalar or aggregate) as SQLite UDFs.
>You can create things that are much more suited to your needs. UDFs can be reused so you don't have to type it again and again. A function can be made in python then applied to SQL tables much easier. I think. Maybe.